In [38]:
import numpy as np
import pandas as pd

In [39]:
import datetime as dt
from datetime import datetime
import time

In [40]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [41]:
engine = create_engine("sqlite:///Resources/SF_BikeShare.sqlite")
conn = engine.connect()

In [42]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [43]:
Base.classes.keys()

['station', 'trip']

In [44]:
# Save references to each table
Trip = Base.classes.trip
Station = Base.classes.station

In [45]:
# Create our session (link) from Python to the DB
session = Session(bind=conn)

In [46]:
inspector = inspect(engine)

In [47]:
columns = inspector.get_columns("station")
for column in columns:
    print(column["name"], column["type"])

id INTEGER
name TEXT
lat NUMERIC
long NUMERIC
dock_count INTEGER
city TEXT
installation_date TEXT


In [48]:
columns = inspector.get_columns("trip")
for column in columns:
    print(column["name"], column["type"])

id INTEGER
duration INTEGER
start_date TEXT
start_station_name TEXT
start_station_id INTEGER
end_date TEXT
end_station_name TEXT
end_station_id INTEGER
bike_id INTEGER
subscription_type TEXT
zip_code INTEGER


In [49]:
# Getting all cities and their zipcodes from a csv file
zipcode_df = pd.read_csv("./Resources/zipcode-database.csv")
zipcode_df = zipcode_df[["Zipcode", "City"]]
zipcode_df.head()

,Zipcode,City
0,705,AIBONITO
1,610,ANASCO
2,611,ANGELES
3,612,ARECIBO
4,601,ADJUNTAS


In [50]:
# Retrieve SF-bike-share data from trip and station tables of sqlite database
sel = [Trip.id.label("trip_id"), Trip.duration, Trip.start_date, Trip.start_station_name, Trip.start_station_id,
       Trip.end_date, Trip.end_station_name, Trip.end_station_id, Trip.bike_id, Trip.subscription_type, Trip.zip_code,
       Station.city.label("start_city"), Station.dock_count.label("start_station_dock_count")
      ]

df_data = session.query(*sel).filter(Trip.start_station_name == Station.name).all()
SF_trip_df = pd.DataFrame(df_data)
SF_trip_df.head()

,trip_id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_city,start_station_dock_count
0,4069,174,8/29/2013 9:08,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,94114,San Francisco,15
1,4073,1067,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:42,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,94703,San Francisco,19
2,4074,1131,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,94115,San Francisco,19
3,4075,1117,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,316,Subscriber,94122,San Francisco,19
4,4076,1118,8/29/2013 9:25,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,322,Subscriber,94597,San Francisco,19


In [51]:
SF_trip_df.count()

trip_id                     658617
duration                    658617
start_date                  658617
start_station_name          658617
start_station_id            658617
end_date                    658617
end_station_name            658617
end_station_id              658617
bike_id                     658617
subscription_type           658617
zip_code                    658617
start_city                  658617
start_station_dock_count    658617
dtype: int64

In [52]:
SF_trip_df["start_city_U"] = SF_trip_df["start_city"].str.upper()
SF_trip_df["start_city_U"].head()

0    SAN FRANCISCO
1    SAN FRANCISCO
2    SAN FRANCISCO
3    SAN FRANCISCO
4    SAN FRANCISCO
Name: start_city_U, dtype: object

In [53]:
SF_trip_df["start_datetime"] = [datetime.strptime(x, "%m/%d/%Y %H:%M") for x in SF_trip_df["start_date"]]
SF_trip_df.head()

,trip_id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_city,start_station_dock_count,start_city_U,start_datetime
0,4069,174,8/29/2013 9:08,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,94114,San Francisco,15,SAN FRANCISCO,2013-08-29 09:08:00
1,4073,1067,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:42,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,94703,San Francisco,19,SAN FRANCISCO,2013-08-29 09:24:00
2,4074,1131,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,94115,San Francisco,19,SAN FRANCISCO,2013-08-29 09:24:00
3,4075,1117,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,316,Subscriber,94122,San Francisco,19,SAN FRANCISCO,2013-08-29 09:24:00
4,4076,1118,8/29/2013 9:25,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,322,Subscriber,94597,San Francisco,19,SAN FRANCISCO,2013-08-29 09:25:00


In [54]:
SF_trip_df["start_time"] = [x.strftime("%H") for x in SF_trip_df["start_datetime"]]
SF_trip_df["start_time"] = pd.to_numeric(SF_trip_df["start_time"])
SF_trip_df.head()

,trip_id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_city,start_station_dock_count,start_city_U,start_datetime,start_time
0,4069,174,8/29/2013 9:08,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,94114,San Francisco,15,SAN FRANCISCO,2013-08-29 09:08:00,9
1,4073,1067,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:42,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,94703,San Francisco,19,SAN FRANCISCO,2013-08-29 09:24:00,9
2,4074,1131,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,94115,San Francisco,19,SAN FRANCISCO,2013-08-29 09:24:00,9
3,4075,1117,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,316,Subscriber,94122,San Francisco,19,SAN FRANCISCO,2013-08-29 09:24:00,9
4,4076,1118,8/29/2013 9:25,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,322,Subscriber,94597,San Francisco,19,SAN FRANCISCO,2013-08-29 09:25:00,9


In [55]:
SF_trip_df["start_date"] = [x.strftime("%Y-%m-%d") for x in SF_trip_df["start_datetime"]]
SF_trip_df.head()

,trip_id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_city,start_station_dock_count,start_city_U,start_datetime,start_time
0,4069,174,2013-08-29,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,94114,San Francisco,15,SAN FRANCISCO,2013-08-29 09:08:00,9
1,4073,1067,2013-08-29,South Van Ness at Market,66,8/29/2013 9:42,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,94703,San Francisco,19,SAN FRANCISCO,2013-08-29 09:24:00,9
2,4074,1131,2013-08-29,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,94115,San Francisco,19,SAN FRANCISCO,2013-08-29 09:24:00,9
3,4075,1117,2013-08-29,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,316,Subscriber,94122,San Francisco,19,SAN FRANCISCO,2013-08-29 09:24:00,9
4,4076,1118,2013-08-29,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,322,Subscriber,94597,San Francisco,19,SAN FRANCISCO,2013-08-29 09:25:00,9


In [56]:
bins = [0, 6, 12, 18, 24]
label_names = ["12am-6am", "6am-12pm", "12pm-6pm", "6pm-12am"]
SF_trip_df["start_time_range"] = pd.cut(SF_trip_df["start_time"], bins, labels=label_names)
SF_trip_df.head()

,trip_id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_city,start_station_dock_count,start_city_U,start_datetime,start_time,start_time_range
0,4069,174,2013-08-29,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,94114,San Francisco,15,SAN FRANCISCO,2013-08-29 09:08:00,9,6am-12pm
1,4073,1067,2013-08-29,South Van Ness at Market,66,8/29/2013 9:42,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,94703,San Francisco,19,SAN FRANCISCO,2013-08-29 09:24:00,9,6am-12pm
2,4074,1131,2013-08-29,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,94115,San Francisco,19,SAN FRANCISCO,2013-08-29 09:24:00,9,6am-12pm
3,4075,1117,2013-08-29,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,316,Subscriber,94122,San Francisco,19,SAN FRANCISCO,2013-08-29 09:24:00,9,6am-12pm
4,4076,1118,2013-08-29,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,322,Subscriber,94597,San Francisco,19,SAN FRANCISCO,2013-08-29 09:25:00,9,6am-12pm


In [57]:
SF_trip_df = SF_trip_df[["trip_id", "start_date", "start_station_name", "end_station_name", "duration", "bike_id",
                 "subscription_type", "start_station_dock_count", "start_city_U", "start_time_range"]]
SF_trip_df.head()

,trip_id,start_date,start_station_name,end_station_name,duration,bike_id,subscription_type,start_station_dock_count,start_city_U,start_time_range
0,4069,2013-08-29,2nd at South Park,2nd at South Park,174,288,Subscriber,15,SAN FRANCISCO,6am-12pm
1,4073,2013-08-29,South Van Ness at Market,San Francisco Caltrain 2 (330 Townsend),1067,321,Subscriber,19,SAN FRANCISCO,6am-12pm
2,4074,2013-08-29,South Van Ness at Market,San Francisco Caltrain 2 (330 Townsend),1131,317,Subscriber,19,SAN FRANCISCO,6am-12pm
3,4075,2013-08-29,South Van Ness at Market,San Francisco Caltrain 2 (330 Townsend),1117,316,Subscriber,19,SAN FRANCISCO,6am-12pm
4,4076,2013-08-29,South Van Ness at Market,San Francisco Caltrain 2 (330 Townsend),1118,322,Subscriber,19,SAN FRANCISCO,6am-12pm


In [58]:
SF_trip_df["route"] = SF_trip_df["start_station_name"] + " TO " + SF_trip_df["end_station_name"] 
SF_trip_df.head(1)

,trip_id,start_date,start_station_name,end_station_name,duration,bike_id,subscription_type,start_station_dock_count,start_city_U,start_time_range,route
0,4069,2013-08-29,2nd at South Park,2nd at South Park,174,288,Subscriber,15,SAN FRANCISCO,6am-12pm,2nd at South Park TO 2nd at South Park


In [59]:
weather_df = pd.read_csv("./Resources/weather.csv")
weather_df.head()

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,8/29/2013,74.0,68.0,61.0,61.0,58.0,56.0,93.0,75.0,57.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
1,8/30/2013,78.0,69.0,60.0,61.0,58.0,56.0,90.0,70.0,50.0,...,10.0,7.0,29.0,13.0,35.0,0,2.0,NaN,291.0,94107
2,8/31/2013,71.0,64.0,57.0,57.0,56.0,54.0,93.0,75.0,57.0,...,10.0,10.0,26.0,15.0,31.0,0,4.0,NaN,284.0,94107
3,9/1/2013,74.0,66.0,58.0,60.0,56.0,53.0,87.0,68.0,49.0,...,10.0,10.0,25.0,13.0,29.0,0,4.0,NaN,284.0,94107
4,9/2/2013,75.0,69.0,62.0,61.0,60.0,58.0,93.0,77.0,61.0,...,10.0,6.0,23.0,12.0,30.0,0,6.0,NaN,277.0,94107


In [60]:
weather_df = weather_df[["date", "max_temperature_f", "mean_temperature_f", "min_temperature_f", "mean_humidity",
                        "mean_visibility_miles", "mean_wind_speed_mph", "precipitation_inches", "cloud_cover", "zip_code"]]

In [61]:
weather_df["date"] = [datetime.strptime(x, "%m/%d/%Y").strftime("%Y-%m-%d") for x in weather_df["date"]]
weather_df.head()

,date,max_temperature_f,mean_temperature_f,min_temperature_f,mean_humidity,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover,zip_code
0,2013-08-29,74.0,68.0,61.0,75.0,10.0,11.0,0,4.0,94107
1,2013-08-30,78.0,69.0,60.0,70.0,10.0,13.0,0,2.0,94107
2,2013-08-31,71.0,64.0,57.0,75.0,10.0,15.0,0,4.0,94107
3,2013-09-01,74.0,66.0,58.0,68.0,10.0,13.0,0,4.0,94107
4,2013-09-02,75.0,69.0,62.0,77.0,10.0,12.0,0,6.0,94107


In [62]:
weather_df = weather_df.merge(zipcode_df, how="inner", left_on="zip_code", right_on="Zipcode")
del weather_df["Zipcode"]
weather_df.rename(columns={"City": "city"}, inplace=True)
weather_df.head()

,date,max_temperature_f,mean_temperature_f,min_temperature_f,mean_humidity,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover,zip_code,city
0,2013-08-29,74.0,68.0,61.0,75.0,10.0,11.0,0,4.0,94107,SAN FRANCISCO
1,2013-08-30,78.0,69.0,60.0,70.0,10.0,13.0,0,2.0,94107,SAN FRANCISCO
2,2013-08-31,71.0,64.0,57.0,75.0,10.0,15.0,0,4.0,94107,SAN FRANCISCO
3,2013-09-01,74.0,66.0,58.0,68.0,10.0,13.0,0,4.0,94107,SAN FRANCISCO
4,2013-09-02,75.0,69.0,62.0,77.0,10.0,12.0,0,6.0,94107,SAN FRANCISCO


In [63]:
merged_df = SF_trip_df.merge(weather_df, how="inner", left_on=["start_date", "start_city_U"], 
                             right_on=["date", "city"])
merged_df.head()

,trip_id,start_date,start_station_name,end_station_name,duration,bike_id,subscription_type,start_station_dock_count,start_city_U,start_time_range,...,max_temperature_f,mean_temperature_f,min_temperature_f,mean_humidity,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover,zip_code,city
0,4069,2013-08-29,2nd at South Park,2nd at South Park,174,288,Subscriber,15,SAN FRANCISCO,6am-12pm,...,74.0,68.0,61.0,75.0,10.0,11.0,0,4.0,94107,SAN FRANCISCO
1,4073,2013-08-29,South Van Ness at Market,San Francisco Caltrain 2 (330 Townsend),1067,321,Subscriber,19,SAN FRANCISCO,6am-12pm,...,74.0,68.0,61.0,75.0,10.0,11.0,0,4.0,94107,SAN FRANCISCO
2,4074,2013-08-29,South Van Ness at Market,San Francisco Caltrain 2 (330 Townsend),1131,317,Subscriber,19,SAN FRANCISCO,6am-12pm,...,74.0,68.0,61.0,75.0,10.0,11.0,0,4.0,94107,SAN FRANCISCO
3,4075,2013-08-29,South Van Ness at Market,San Francisco Caltrain 2 (330 Townsend),1117,316,Subscriber,19,SAN FRANCISCO,6am-12pm,...,74.0,68.0,61.0,75.0,10.0,11.0,0,4.0,94107,SAN FRANCISCO
4,4076,2013-08-29,South Van Ness at Market,San Francisco Caltrain 2 (330 Townsend),1118,322,Subscriber,19,SAN FRANCISCO,6am-12pm,...,74.0,68.0,61.0,75.0,10.0,11.0,0,4.0,94107,SAN FRANCISCO


In [64]:
del merged_df["date"]
del merged_df["zip_code"]
del merged_df["city"]

In [65]:
merged_df.count()

trip_id                     658617
start_date                  658617
start_station_name          658617
end_station_name            658617
duration                    658617
bike_id                     658617
subscription_type           658617
start_station_dock_count    658617
start_city_U                658617
start_time_range            656493
route                       658617
max_temperature_f           658551
mean_temperature_f          658551
min_temperature_f           658551
mean_humidity               658264
mean_visibility_miles       658464
mean_wind_speed_mph         658603
precipitation_inches        658603
cloud_cover                 658603
dtype: int64

In [66]:
merged_df["start_date"].max()

'2015-08-31'

In [67]:
merged_df["start_date"].min()

'2013-08-29'

In [68]:
# Geeting only one year of recent data
reduced_merged_df = merged_df.loc[merged_df["start_date"]>"2014-08-31"]

In [69]:
reduced_merged_df.to_csv("SF_BikeShare_data.csv", index=False)

In [70]:
reduced_merged_df.count()

trip_id                     346787
start_date                  346787
start_station_name          346787
end_station_name            346787
duration                    346787
bike_id                     346787
subscription_type           346787
start_station_dock_count    346787
start_city_U                346787
start_time_range            345809
route                       346787
max_temperature_f           346721
mean_temperature_f          346721
min_temperature_f           346721
mean_humidity               346476
mean_visibility_miles       346734
mean_wind_speed_mph         346773
precipitation_inches        346773
cloud_cover                 346773
dtype: int64

In [71]:
SF_groupby = merged_df.groupby(["start_date", "start_time_range", "route", "subscription_type"])
SF_agg = SF_groupby.agg({"trip_id": "count", "duration":"mean"})
SF_agg.head()

trip_id  \
start_date start_time_range route                                              subscription_type            
2013-08-29 6am-12pm         2nd at Folsom TO 2nd at South Park                 Subscriber               1   
                            2nd at South Park TO 2nd at South Park             Subscriber               2   
                            2nd at Townsend TO 2nd at South Park               Customer                 1   
                            2nd at Townsend TO Embarcadero at Folsom           Customer                 1   
                            2nd at Townsend TO San Francisco Caltrain 2 (33... Subscriber               1   

                                                                                                  duration  
start_date start_time_range route                                              subscription_type            
2013-08-29 6am-12pm         2nd at Folsom TO 2nd at South Park                 Subscriber            825.0  
                            2nd at South Park TO 2nd at South Park             Subscriber            578.0  
                            2nd at Townsend TO 2nd at South Park               Customer              173.0  
                            2nd at Townsend TO Embarcadero at Folsom           Customer             1776.0  
                            2nd at Townsend TO San Francisco Caltrain 2 (33... Subscriber            223.0